<a href="https://colab.research.google.com/github/Imrantipu/-serviceReviewServer/blob/main/optuna_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy insted of .TPESampler we can use .RandomSampler() for other sampling
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-01-31 08:23:23,823] A new study created in memory with name: no-name-f053c8a6-fa04-477e-badb-6e4acb970fad
[I 2025-01-31 08:23:26,019] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 115, 'max_depth': 9}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-01-31 08:23:26,829] Trial 1 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 101, 'max_depth': 3}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-01-31 08:23:27,753] Trial 2 finished with value: 0.7541899441340782 and parameters: {'n_estimators': 179, 'max_depth': 3}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-01-31 08:23:28,693] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 168, 'max_depth': 16}. Best is trial 3 with value: 0.7765363128491621.
[I 2025-01-31 08:23:29,366] Trial 4 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 121, 'max_depth': 9}. Best is trial 3 with value: 0.77653631

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 19}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


# Samplers in optuna

In [9]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [10]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-01-31 08:41:47,858] A new study created in memory with name: no-name-644009cf-03ab-4bff-8cee-16d31ddbb3cf
[I 2025-01-31 08:41:48,388] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-31 08:41:49,227] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-31 08:41:49,525] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-31 08:41:50,107] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-31 08:41:50,668] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


# Optuna visualization

In [13]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [14]:
# 1. Optimization History
plot_optimization_history(study).show()

In [15]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [16]:
# 3. Slice Plot
plot_slice(study).show()

In [17]:
# 4. Contour Plot
plot_contour(study).show()

In [18]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

#Optimization Multiple ML Models

In [19]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [20]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [21]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-01-31 08:58:17,053] A new study created in memory with name: no-name-d2eb390c-194a-4c04-b3de-fd936d1ae858
[I 2025-01-31 08:58:18,189] Trial 0 finished with value: 0.750465549348231 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 73, 'learning_rate': 0.19941175183688192, 'max_depth': 8, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.750465549348231.
[I 2025-01-31 08:58:18,224] Trial 1 finished with value: 0.7374301675977654 and parameters: {'classifier': 'SVM', 'C': 2.695816639577965, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.750465549348231.
[I 2025-01-31 08:58:18,255] Trial 2 finished with value: 0.7374301675977654 and parameters: {'classifier': 'SVM', 'C': 0.38248374654782724, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.750465549348231.
[I 2025-01-31 08:58:18,770] Trial 3 finished with value: 0.7728119180633147 and parameters: {'classifier': 'RandomForest', 'n_estimators': 117, '

In [22]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1204798367776354, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [23]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.750466,2025-01-31 08:58:17.056356,2025-01-31 08:58:18.189099,0 days 00:00:01.132743,NaN,NaN,GradientBoosting,NaN,NaN,0.199412,8.0,2.0,3.0,73.0,COMPLETE
1,1,0.737430,2025-01-31 08:58:18.190909,2025-01-31 08:58:18.224697,0 days 00:00:00.033788,2.695817,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.737430,2025-01-31 08:58:18.226440,2025-01-31 08:58:18.254871,0 days 00:00:00.028431,0.382484,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.772812,2025-01-31 08:58:18.256749,2025-01-31 08:58:18.769956,0 days 00:00:00.513207,NaN,False,RandomForest,NaN,NaN,NaN,15.0,8.0,7.0,117.0,COMPLETE
4,4,0.785847,2025-01-31 08:58:18.771661,2025-01-31 08:58:18.939934,0 days 00:00:00.168273,31.709567,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-01-31 08:59:06.666620,2025-01-31 08:59:06.701282,0 days 00:00:00.034662,0.223402,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.743017,2025-01-31 08:59:06.702972,2025-01-31 08:59:06.743665,0 days 00:00:00.040693,0.119055,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.746741,2025-01-31 08:59:06.745252,2025-01-31 08:59:09.858883,0 days 00:00:03.113631,NaN,NaN,GradientBoosting,NaN,NaN,0.020902,16.0,3.0,3.0,146.0,COMPLETE
98,98,0.789572,2025-01-31 08:59:09.860608,2025-01-31 08:59:09.897229,0 days 00:00:00.036621,0.147951,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [24]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
GradientBoosting,12
RandomForest,9


In [25]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.747207
RandomForest,0.761639
SVM,0.774910


In [26]:
# 1. Optimization History
plot_optimization_history(study).show()

In [27]:
# 3. Slice Plot
plot_slice(study).show()

In [28]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [30]:
!pip install optuna-integration[xgboost]
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.4 MB/s eta 0:00:00


[I 2025-01-31 09:12:20,197] A new study created in memory with name: no-name-97f055a5-5a4c-4561-9588-f394cb7ea03d


[0]	train-mlogloss:0.85916	eval-mlogloss:0.84934
[1]	train-mlogloss:0.62438	eval-mlogloss:0.60287
[2]	train-mlogloss:0.46796	eval-mlogloss:0.43674
[3]	train-mlogloss:0.35864	eval-mlogloss:0.32125
[4]	train-mlogloss:0.28547	eval-mlogloss:0.24397
[5]	train-mlogloss:0.22998	eval-mlogloss:0.18061
[6]	train-mlogloss:0.18965	eval-mlogloss:0.14548
[7]	train-mlogloss:0.16375	eval-mlogloss:0.12161
[8]	train-mlogloss:0.14570	eval-mlogloss:0.09886
[9]	train-mlogloss:0.13546	eval-mlogloss:0.08635
[10]	train-mlogloss:0.13243	eval-mlogloss:0.08434
[11]	train-mlogloss:0.12775	eval-mlogloss:0.07512
[12]	train-mlogloss:0.12650	eval-mlogloss:0.07591
[13]	train-mlogloss:0.12349	eval-mlogloss:0.06958
[14]	train-mlogloss:0.12235	eval-mlogloss:0.07168
[15]	train-mlogloss:0.12011	eval-mlogloss:0.06578
[16]	train-mlogloss:0.11742	eval-mlogloss:0.06224
[17]	train-mlogloss:0.11721	eval-mlogloss:0.06190
[18]	train-mlogloss:0.11586	eval-mlogloss:0.05990
[19]	train-mlogloss:0.11538	eval-mlogloss:0.05966
[20]	train

[I 2025-01-31 09:12:20,642] Trial 0 finished with value: 1.0 and parameters: {'lambda': 5.462384910216173e-06, 'alpha': 1.058782621859015e-06, 'eta': 0.2901641528553862, 'gamma': 4.0183289378553344e-08, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6360174671160264, 'colsample_bytree': 0.6587588732736163}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.08860	eval-mlogloss:1.08820
[1]	train-mlogloss:1.07578	eval-mlogloss:1.07410
[2]	train-mlogloss:1.06124	eval-mlogloss:1.05896
[3]	train-mlogloss:1.04648	eval-mlogloss:1.04390
[4]	train-mlogloss:1.03259	eval-mlogloss:1.02988
[5]	train-mlogloss:1.01837	eval-mlogloss:1.01504
[6]	train-mlogloss:1.00500	eval-mlogloss:1.00116
[7]	train-mlogloss:0.99359	eval-mlogloss:0.99061
[8]	train-mlogloss:0.98075	eval-mlogloss:0.97689
[9]	train-mlogloss:0.96790	eval-mlogloss:0.96353
[10]	train-mlogloss:0.95566	eval-mlogloss:0.95103
[11]	train-mlogloss:0.94349	eval-mlogloss:0.93792
[12]	train-mlogloss:0.93550	eval-mlogloss:0.92973
[13]	train-mlogloss:0.92356	eval-mlogloss:0.91683
[14]	train-mlogloss:0.91615	eval-mlogloss:0.90886
[15]	train-mlogloss:0.90546	eval-mlogloss:0.89797
[16]	train-mlogloss:0.89554	eval-mlogloss:0.88761
[17]	train-mlogloss:0.88664	eval-mlogloss:0.87874
[18]	train-mlogloss:0.87547	eval-mlogloss:0.86723
[19]	train-mlogloss:0.86654	eval-mlogloss:0.85828
[20]	train

...

[0]	train-mlogloss:1.05318	eval-mlogloss:1.05062
[1]	train-mlogloss:1.00685	eval-mlogloss:0.99747


[I 2025-01-31 09:12:32,102] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:0.89373	eval-mlogloss:0.88462


[I 2025-01-31 09:12:32,267] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 5.462384910216173e-06, 'alpha': 1.058782621859015e-06, 'eta': 0.2901641528553862, 'gamma': 4.0183289378553344e-08, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6360174671160264, 'colsample_bytree': 0.6587588732736163}
Best accuracy: 1.0


In [31]:
! pip install optuna-integration[xgboost]

In [32]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()